In [ ]:
import yaml

from linkml.utils.schema_builder import SchemaBuilder
from linkml_runtime.linkml_model import SlotDefinition


In [ ]:
sb = SchemaBuilder()
sb.add_class("Person", slots=[SlotDefinition("family_name", range="string"), 
                              SlotDefinition("given_name", range="string"),
                              SlotDefinition("age_in_years", range="integer"),
                              SlotDefinition("height_in_cm", range="float"),
                              ])
sb.add_defaults()
print(yaml.dump(sb.as_dict(), sort_keys=False))


In [ ]:
from linkml_map.session import Session

session = Session()
session.set_source_schema(sb.as_dict())

In [ ]:
# Transformer specification (in YAML)
session.set_object_transformer("""
class_derivations:
  Individual:
    populated_from: Person
    slot_derivations:
      family_name:
        populated_from: family_name
      given_name:
        populated_from: given_name
      age:
        populated_from: age_in_years
      height:
        populated_from: height_in_cm
""")

obj = {
        "given_name": "Jane",
        "family_name": "Doe",
        "age_in_years": 42,
        "height_in_cm": 180.0,
    }
session.transform(obj)

In [ ]:
from linkml_runtime.dumpers import yaml_dumper

print(yaml_dumper.dumps(session.target_schema))

In [ ]:
session.set_object_transformer("""
class_derivations:
  Individual:
    populated_from: Person
    slot_derivations:
      name:
        expr: "{given_name} + ' ' + {family_name}"
        description: Concatenating given and family names
          note this is a bad assumption for names in general,
          this is just for demonstration
      age_in_months:
        expr: age_in_years * 12
      height_in_meters:
        expr: height_in_cm / 100
""")

session.transform(obj)

In [ ]:
print(yaml_dumper.dumps(session.transform(obj)))